In [5]:
import pandas as pd
import numpy as np

# modulos de ayuda
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

# leemos la informacion
professors = pd.read_csv("data/pakistan_intellectual_capital.csv")

# configuramos la semilla
np.random.seed(0)

In [9]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [17]:
# obtengamos los valores unicos por pai
countries = list(professors['Country'].unique())

# obtenemos los datos unicos y los ordenamos
countries.sort()
countries

[' Germany',
 ' New Zealand',
 ' Sweden',
 ' USA',
 'Australia',
 'Austria',
 'Canada',
 'China',
 'Finland',
 'France',
 'Greece',
 'HongKong',
 'Ireland',
 'Italy',
 'Japan',
 'Macau',
 'Malaysia',
 'Mauritius',
 'Netherland',
 'New Zealand',
 'Norway',
 'Pakistan',
 'Portugal',
 'Russian Federation',
 'Saudi Arabia',
 'Scotland',
 'Singapore',
 'South Korea',
 'SouthKorea',
 'Spain',
 'Sweden',
 'Thailand',
 'Turkey',
 'UK',
 'USA',
 'USofA',
 'Urbana',
 'germany']

# Hay errores
Vemos que en el dataset hay datos inconsistentes, es decir, tenemos un conjunto de datos que no estan bien registrados. Podemos ver que hay valores que tienen un espacio por delante y que hay valores que se duplican, aunque no lo hacen ya que si hay mas de un valor string posiblemente tenga un error en su ortografia, como 'South Korea" y 'SouthKorea'.

In [20]:
# arreglemos los datos de entrada de aquellos paises con mayuscualas, para mayor comodidad nuestra
# los convertiremos a lower.
professors['Country'] = professors['Country'].str.lower()

# ahora, eliminamos los espacios en blanco
professors['Country'] = professors['Country'].str.strip()
countries

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [24]:
countries = list(professors['Country'].unique())
countries.sort()
countries

['australia',
 'austria',
 'canada',
 'china',
 'finland',
 'france',
 'germany',
 'greece',
 'hongkong',
 'ireland',
 'italy',
 'japan',
 'macau',
 'malaysia',
 'mauritius',
 'netherland',
 'new zealand',
 'norway',
 'pakistan',
 'portugal',
 'russian federation',
 'saudi arabia',
 'scotland',
 'singapore',
 'south korea',
 'southkorea',
 'spain',
 'sweden',
 'thailand',
 'turkey',
 'uk',
 'urbana',
 'usa',
 'usofa']

Vemos que ahora tenemos menos unas cuantas menos entradas, pero, aun no corregimos algunos errores. Aquellas entradas que ya no tiene el espacio enfrente de ellso ya no existen, pero aun se conservan las que tienen espacios entre palabras, ademas, tambien vemos que no hemos eliminado los duplicados. Supongamos lo siguiente, si quicieramos buscar cuantas veces 'South Korea' se ha registrado mal, y ademas nuestro dataset contien mas paises de los que tenemos aqui, Comparar todas las entradas aunque sea en una lista nos tomaria demaciado tiempo, para evita esto utilizaremos la libreria fuzzywuzzy.
# Como funciona fuzzywuzzy
fuzzy wuzzy trabaja con porcentajes de relacion, es decir, creando una funcion a la que le pediremos analizar, fuzzywuzzy nos retornara una lista con el porcentaje de relacion que cada palabra tiene con la palabra que le pedimos analizar.

In [32]:
countries = professors['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [34]:
matches = fuzzywuzzy.process.extract("south korea", countries, limit = 10, scorer = fuzzywuzzy.fuzz.token_sort_ratio)

matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

In [52]:
# vemos ahora unal lista de tuplas dobles donde tenemos el nombre de los paises y su respectivo
# porcentaje de relacion.
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    # conseguimos al menos los 10 resultados mas cercanos a sting de entrada que queremos
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit = 10, 
                                         scorer = fuzzywuzzy.fuzz.token_sort_ratio)
    # solo tomamos los emparejamientos con ratio >90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    # esta funcion toma el primer elemento de cada tupla (matches[0] de la lista de matches
    # que cumple con la condicion de aquellos elementos del la lista matches de matches[1] >= min_ratio
    rows_with_matches = df[column].isin(close_matches)
    # remplaza todas las filas con los matches mas cercanos al input
    df.loc[rows_with_matches, column] = string_to_match
    # ahora hay que saber si todo esta bien
    print("All done")
    

In [54]:
# Usemos la funcion  para saber que funciones se remplazaran con el 'south korea' correctamente
replace_matches_in_column(df = professors, column = 'Country', string_to_match = "south korea")

All done


In [56]:
countries = professors['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Y ahora ya no tenemos la entrada soutkorea, si hacemos lo mismo con los demas sterings podremos corregir el dataset completo.
